In [1]:
from random import randint

In [2]:
# Windows only
# gpt2_tuned_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\2.ModelingGPT2andXLNet\testing_data_run\combined_data\test_models_results.txt"
# gpt2_untuned_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\2.ModelingGPT2andXLNet\testing_data_run\combined_data\test_gpt2_results.txt"
# xlnet_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\2.ModelingGPT2andXLNet\testing_data_run\combined_data\test_xlnet-base-cased_results.txt"
# lstm_attention_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\LSTM-word\textgenrnn2\LSTM-attention-result-test-fixed-30-epoch.txt"
# gpt2_untuned_bert_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\BERT_HAMMERING\gpt2_BERT_result-new-dim.txt"

gpt2_tuned_path = r"../2.ModelingGPT2andXLNet/testing_data_run/combined_data/test_models_results.txt"
gpt2_untuned_path = r"../2.ModelingGPT2andXLNet/testing_data_run/combined_data/test_gpt2_results.txt"
xlnet_path = r"../2.ModelingGPT2andXLNet/testing_data_run/combined_data/test_xlnet-base-cased_results.txt"
lstm_attention_path = r"../LSTM-word/textgenrnn2/LSTM-attention-result-test-fixed-30-epoch.txt"
gpt2_untuned_bert_path = r"../BERT_HAMMERING/gpt2_BERT_result-new-dim.txt"

In [3]:
max_line_review = 2700

# Windows only
# output1_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\survey\question_prep1.txt"
# output2_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\survey\question_prep2.txt"
# output3_path = r"D:\UCBerkeley\CourseWork\201909\W266\GitHub\FinalProject-Collab\survey\question_prep3.txt"

output1_path = r"../survey/question_prep1.txt"
# output2_path = r"../survey/question_prep2.txt"
# output3_path = r"../survey/question_prep3.txt"

In [4]:
gpt2_tuned_file = open(gpt2_tuned_path, 'r', encoding="utf8").read().split('<end_line>')
gpt2_untuned_file = open(gpt2_untuned_path, 'r', encoding="utf8").read().split('<end_line>')
xlnet_file = open(xlnet_path, 'r', encoding="utf8").read().split('<end_line>')
lstm_attention_file = open(lstm_attention_path, 'r', encoding="utf8").read().split('<end_line>')
gpt2_untuned_bert_file = open(gpt2_untuned_bert_path, 'r', encoding="utf8").read().split('<end_line>')

In [5]:
def check_repeats(file, verbose=1, max_line=None):
    
    # flag as repeats if {repeat_threshold}% of test input is in the generated sentence
    repeat_threshold = 0.5
    
    repeat_count = 0
    invalid_count = 0
    special_character_count = 0
    valid_sentence_index = []
    input_set = set()
    input_duplicate_set = set()
    
    for i in range(0, len(file)):
        
        if max_line is not None:
            if i == max_line:
                break
                
        line = file[i].split('<entry>')
        duplicate_binary = 0
        
        # Make sure it has input_sentence, actual_sentence, and pred_sentence
        if len(line) < 3:
            invalid_count += 1
            continue
            
        if line[0][0:10] != '<new_line>':
            invalid_count += 1   
            continue
            
        # sentences less than 30 characters are invalid
        if min(len(line[0][10:]), len(line[1]), len(line[2])) <= 30:
            invalid_count += 1
            continue  
            
        input_sentence = line[0][10:]
        # Create a set of duplicate inputs
        if input_sentence not in input_set:
            input_set.add(input_sentence)
        else:
            input_duplicate_set.add(input_sentence)
            
        actual_sentence = line[1]
        pred_sentence = line[2]
        
        # Don't want carriage return, # sign, or excessive quotes etc. in the predicted sentence
        if '\n' in pred_sentence or '#' in pred_sentence or '("")' in pred_sentence or '()' in pred_sentence \
                or '"' in pred_sentence:
            special_character_count += 1
            continue  
        
        input_len = len(input_sentence.split())
        check_len = round(input_len*repeat_threshold)
    
        for start_index in range(0, input_len-check_len):
            check_list = input_sentence.split()[start_index: start_index + check_len]
            check_phrase = ' '.join(check_list)
            if check_phrase in pred_sentence:
                if verbose == 1:
                    print(f"-------- line {i} --------")
                    print(input_sentence)
                    print(pred_sentence)
                repeat_count += 1
                duplicate_binary = 1
                break
        
        if duplicate_binary == 0:
            valid_sentence_index.append(i)
        
    print('Number of invalid sentences: ', invalid_count)
    print('Number of generated sentences similar to the input sentence: ', repeat_count)
    print('Number of input sentences that appear multiple times: ', len(input_duplicate_set))
    print('Number of sentences containing carriage return, # signs, or excessive quotes etc.: ', special_character_count)
    if max_line is not None:
        print('Sanity checks above are based off of the first ', max_line, 'lines. ')
    print('Total number of input sentences: ', len(file))
    
    return valid_sentence_index, input_duplicate_set

In [6]:
def check_repeats2(file, verbose=1, max_line=None):
    """
    This function only uses for gpt2_untuned_bert because the data format is different
    :param file: 
    :param verbose: 
    :param max_line: 
    :return: 
    """
    # flag as repeats if {repeat_threshold}% of test input is in the generated sentence
    repeat_threshold = 0.5

    repeat_count = 0
    invalid_count = 0
    special_character_count = 0
    valid_sentence_index = []
    input_set = set()
    input_duplicate_set = set()
    
    for i in range(0, len(file)):
        
        if max_line is not None:
            if i == max_line:
                break
                
        line = file[i].split('<entry>')
        duplicate_binary = 0
        
        if len(line) < 7:
            invalid_count += 1
            continue
            
        if line[0][0:10] != '<new_line>':
            invalid_count += 1   
            continue
        # sentences less than 30 characters are invalid
        if min(len(line[0][10:]), len(line[1]), len(line[3])) <= 30:
            invalid_count += 1
            continue  
            
        input_sentence = line[0][10:]
        # Create a set of duplicate inputs
        if input_sentence not in input_set:
            input_set.add(input_sentence)
        else:
            input_duplicate_set.add(input_sentence)
            
        actual_sentence = line[1]
        pred_sentence = line[3]
        
        # Don't want carriage return, # sign, or excessive quotes etc. in the predicted sentence
        if '\n' in pred_sentence or '#' in pred_sentence or '("")' in pred_sentence or '()' in pred_sentence \
                or '"' in pred_sentence:
            special_character_count += 1
            continue  
            
        input_len = len(input_sentence.split())
        check_len = round(input_len*repeat_threshold)
    
        for start_index in range(0, input_len-check_len):
            check_list = input_sentence.split()[start_index: start_index + check_len]
            check_phrase = ' '.join(check_list)
            if check_phrase in pred_sentence:
                if verbose == 1:
                    print(f"-------- line {i} --------")
                    print(input_sentence)
                    print(pred_sentence)
                repeat_count += 1
                duplicate_binary = 1
                break
        
        if duplicate_binary == 0:
            valid_sentence_index.append(i)
        
    print('Number of invalid sentences: ', invalid_count)
    print('Number of generated sentences similar to the input sentence: ', repeat_count)
    print('Number of input sentences that appear multiple times: ', len(input_duplicate_set))
    print('Number of sentences containing carriage return, # signs, or excessive quotes etc.: ', special_character_count)
    if max_line is not None:
        print('Sanity checks above are based off of the first ', max_line, 'lines. ')
    print('Total number of input sentences: ', len(file))
    
    return valid_sentence_index, input_duplicate_set

In [7]:
print('------------- gpt2 tuned -------------')
gpt2_tuned_valid_sentence_index, duplicate_input1 = check_repeats(gpt2_tuned_file, verbose=0, max_line=max_line_review)
print('------------- gpt2 untuned -------------')
gpt2_untuned_valid_sentence_index, duplicate_input2 = check_repeats(gpt2_untuned_file, verbose=0, max_line=max_line_review)
print('------------- xlnet -------------')
xlnet_valid_sentence_index, duplicate_input3 = check_repeats(xlnet_file, verbose=0, max_line=max_line_review)
print('------------- lstm attention -------------')
lstm_attention_valid_sentence_index, duplicate_input4 = check_repeats(lstm_attention_file, verbose=0, max_line=max_line_review)
print('------------- gpt2 untuned with bert -------------')
gpt2_untuned_bert_valid_sentence_index, duplicate_input5 = check_repeats2(gpt2_untuned_bert_file, verbose=0, max_line=max_line_review)

------------- gpt2 tuned -------------
Number of invalid sentences:  269
Number of generated sentences similar to the input sentence:  1332
Number of input sentences that appear multiple times:  0
Number of sentences containing carriage return, # signs, or excessive quotes etc.:  0
Sanity checks above are based off of the first  2700 lines. 
Total number of input sentences:  12501
------------- gpt2 untuned -------------
Number of invalid sentences:  299
Number of generated sentences similar to the input sentence:  60
Number of input sentences that appear multiple times:  1
Number of sentences containing carriage return, # signs, or excessive quotes etc.:  249
Sanity checks above are based off of the first  2700 lines. 
Total number of input sentences:  4999
------------- xlnet -------------
Number of invalid sentences:  325
Number of generated sentences similar to the input sentence:  131
Number of input sentences that appear multiple times:  1
Number of sentences containing carriage 

In [8]:
duplicate_input_all = duplicate_input1.union(duplicate_input2, duplicate_input3, duplicate_input4, duplicate_input5)

print(len(duplicate_input_all))

1


In [9]:
index =2185
print('------------- gpt2 tuned -------------')
print(gpt2_tuned_file[index])
print('------------- gpt2 untuned -------------')
print(gpt2_untuned_file[index])
print('------------- xlnet -------------')
print(xlnet_file[index])
print('------------- lstm attention -------------')
print(lstm_attention_file[index])
print('------------- gpt2 untuned with bert -------------')
print(gpt2_untuned_bert_file[index])

------------- gpt2 tuned -------------
<new_line>There are cynics , there are skeptics who say it cannot be done.<entry>The American people and all the freedom loving peoples of this earth are now demanding that it must be done.<entry>But I think that the way we do it is to make sure we do it.
------------- gpt2 untuned -------------
<new_line>There are cynics , there are skeptics who say it cannot be done.<entry>The American people and all the freedom loving peoples of this earth are now demanding that it must be done.<entry>But the fact is, we are doing this.
------------- xlnet -------------
<new_line>There are cynics , there are skeptics who say it cannot be done.<entry>The American people and all the freedom loving peoples of this earth are now demanding that it must be done.<entry>There are " ", there are " " and there are " " and there are " " and there are " " and there are " " and there are " " and there are " " and there are " " and there are " " and there are.
------------- 

In [10]:
# test_duplicate_lst = []
# for j in range(50):
#     test_input = list(duplicate_input4)[j]
#     for i, sentences in enumerate(lstm_attention_file):
#         test_input_sentence = sentences.split('<entry>')[0][10:]
#         if test_input == test_input_sentence:
#             test_duplicate_lst.append(i)
#             print(i)
# print(len(test_duplicate_lst))

In [11]:
# Remove gpt2_untuned_bert_valid_sentence_index since it doesn't have the same index as others
valid_sentence_index_union = gpt2_tuned_valid_sentence_index \
                            + gpt2_untuned_valid_sentence_index \
                            + xlnet_valid_sentence_index \
                            + lstm_attention_valid_sentence_index

valid_sentence_index_dict = dict()
for index in valid_sentence_index_union:
    valid_sentence_index_dict[index] = valid_sentence_index_dict.get(index, 0) + 1


In [12]:
valid_sentence_index_intersect = [key for key in valid_sentence_index_dict.keys() if valid_sentence_index_dict[key]==4]
print(len(valid_sentence_index_intersect))

573


In [13]:
# Create a list of valid input sentences based on gpt2_tuned, xlnet, and lstm_attention
input_dict = {sentences.split('<entry>')[0][10:]: i for i, sentences in enumerate(gpt2_tuned_file) if i in valid_sentence_index_intersect}

In [14]:
def match_by_input(file, valid_sentence_index, in_input):
    out_input = dict()
    for i, sentences in enumerate(file):
        input_sentence = sentences.split('<entry>')[0][10:]
        if i in valid_sentence_index and input_sentence in in_input.keys() and input_sentence not in duplicate_input_all:
            out_input[input_sentence] = in_input[input_sentence]
        
    return out_input

input_dict3 = match_by_input(file=gpt2_untuned_bert_file, valid_sentence_index=gpt2_untuned_bert_valid_sentence_index, in_input=input_dict)
# input_dict3 = match_by_input(file=lstm_attention_file, valid_sentence_index=lstm_attention_valid_sentence_index, in_input=input_dict2)

print(len(input_dict3))


440


In [15]:
# Input | actual | gpt2_tuned | xlnet | lstm_attention | gpt2_untuned_bert | gpt2_untuned 
input_index_lst = [k for k in input_dict3.values()]
input_lst = [k for k in input_dict3.keys()]
output_lst = [sentences.split('<entry>')[1] for i, sentences in enumerate(gpt2_tuned_file) if i in input_dict3.values()]
gpt2_tuned_lst = [sentences.split('<entry>')[2] for i, sentences in enumerate(gpt2_tuned_file) if i in input_dict3.values()]
gpt2_untuned_lst = [sentences.split('<entry>')[2] for i, sentences in enumerate(gpt2_untuned_file) if i in input_dict3.values()]
xlnet_lst = [sentences.split('<entry>')[2] for i, sentences in enumerate(xlnet_file) if i in input_dict3.values()]
lstm_attention_lst = [sentences.split('<entry>')[2] for i, sentences in enumerate(lstm_attention_file) if i in input_dict3.values()]

gpt2_untuned_bert_lst = [sentences.split('<entry>')[3] for sentences in gpt2_untuned_bert_file if sentences.split('<entry>')[0][10:] in input_dict3.keys()]

# sanity check
print(len(input_index_lst))
print(len(input_lst))
print(len(output_lst))
print(len(gpt2_tuned_lst))
print(len(xlnet_lst))
print(len(lstm_attention_lst))
print(len(gpt2_untuned_bert_lst))
print(len(gpt2_untuned_lst))

440
440
440
440
440
440
440
440


In [16]:
# Input | actual | gpt2_tuned | xlnet | lstm_attention | gpt2_untuned_bert | gpt2_untuned 
all_data = zip(input_index_lst, input_lst, output_lst, gpt2_tuned_lst, xlnet_lst, lstm_attention_lst, gpt2_untuned_bert_lst, gpt2_untuned_lst)

test = 0
result = open(output1_path, 'w')
# result.write("input_index|input|output|gpt2_tuned|xlnet|lstm_attention|gpt2_untuned_bert|gpt2_untuned\n")
result.write(u"input_index<entry>input<entry>output<entry>gpt2_tuned<entry>xlnet<entry>lstm_attention<entry>gpt2_untuned_bert<entry>gpt2_untuned\n")
for data in all_data:
    data = list(map(str, data))
    result.write(u"<entry>".join(data) + '\n')
    test += 1


In [55]:
manually_discarded_lst = [109, 144, 241, 282, 423, 427, 476, 595, 669, 755, 802, 819, 920, 944, 1022, 1056, 1080, 1116, 1136, 1198, 1329, 1343, 1443, 1497, 1625, 1668, 1760, 1798, 1904, 2036, 2103, 2185, 2257, 2383, 2473, 2522, 2625]

In [56]:
result2 = open(output2_path, 'w')
survey_index_cum_lst = []
for i in range(1000):
    selected_index = randint(0, len(input_index_lst)-1)
    
    if input_index_lst[selected_index] in manually_discarded_lst:
        continue
    elif input_index_lst[selected_index] in survey_index_cum_lst:
        continue
    else:
        survey_index = 50 + selected_index
        survey_index_cum_lst.append(survey_index)
    
    survey_question_p1 = '{"SurveyID":"BL_eCE6nlVo6ilQDCl","Element":"SQ","PrimaryAttribute":"QID'
    survey_question_p2 = str(survey_index)
    survey_question_p3 =  '","SecondaryAttribute":"'
    input_plus_index = str(input_index_lst[selected_index])+'. '+input_lst[selected_index]
    if len(input_plus_index)<=97:
        input_plus_index97 = input_plus_index
    else:
        input_plus_index97 = input_plus_index[:97]+'...'
    survey_question_p4 = input_plus_index97
    survey_question_p5 = '","TertiaryAttribute":null,"Payload":{"QuestionText":"<p>'
    survey_question_p6 = input_plus_index
    survey_question_p7 = '<\/p>","DefaultChoices":false,"DataExportTag":"Q'
    survey_question_p8 = str(survey_index)
    survey_question_p9 = '","QuestionID":"QID'
    survey_question_p10 = str(survey_index)
    survey_question_p11 = '","QuestionType":"Matrix","Selector":"Likert","SubSelector":"DL","Configuration":{"QuestionDescriptionOption":"UseText","TextPosition":"inline","ChoiceColumnWidth":25,"MobileFirst":false},"QuestionDescription":"'
    survey_question_p12 = input_plus_index97
    survey_question_p13 = '","Choices":{"1":{"Display":"'
    survey_question_p14 = output_lst[selected_index]
    survey_question_p15 = '","ExclusiveAnswer":false},"2":{"Display":"'
    survey_question_p16 = gpt2_tuned_lst[selected_index]
    survey_question_p17 = '","ExclusiveAnswer":false},"3":{"Display":"'
    survey_question_p18 = xlnet_lst[selected_index]
    survey_question_p19 = '","ExclusiveAnswer":false},"12":{"Display":"'
    survey_question_p20 = lstm_attention_lst[selected_index]
    survey_question_p21 = '","ExclusiveAnswer":false},"13":{"Display":"'
    survey_question_p22 = gpt2_untuned_bert_lst[selected_index]
    survey_question_p23 = '","ExclusiveAnswer":false},"14":{"Display":"'
    survey_question_p24 = gpt2_untuned_lst[selected_index]
    survey_question_p25 = '","ExclusiveAnswer":false}},"ChoiceOrder":["1","2","3","12","13","14"],"Validation":{"Settings":{"ForceResponse":"ON","ForceResponseType":"ON","Type":"None"}},"GradingData":[],"Language":[],"NextChoiceId":15,"NextAnswerId":9,"Answers":{"1":{"Display":"Terrible"},"2":{"Display":"Bad"},"6":{"Display":"OK"},"7":{"Display":"Good"},"8":{"Display":"Great"}},"AnswerOrder":["1","2","6","7","8"],"ChoiceDataExportTags":false,"Randomization":{"Advanced":null,"Type":"All","TotalRandSubset":""},"DataVisibility":{"Private":false,"Hidden":false},"AnswerColumns":5}},'

    survey_question_x = \
        survey_question_p1\
        +survey_question_p2\
        +survey_question_p3\
        +survey_question_p4\
        +survey_question_p5\
        +survey_question_p6\
        +survey_question_p7\
        +survey_question_p8\
        +survey_question_p9\
        +survey_question_p10\
        +survey_question_p11\
        +survey_question_p12\
        +survey_question_p13\
        +survey_question_p14\
        +survey_question_p15\
        +survey_question_p16\
        +survey_question_p17\
        +survey_question_p18\
        +survey_question_p19\
        +survey_question_p20\
        +survey_question_p21\
        +survey_question_p22\
        +survey_question_p23\
        +survey_question_p24\
        +survey_question_p25
    
    result2.write(survey_question_x)
    
    if len(survey_index_cum_lst) == 30:
        break

result2.close()

In [57]:
result3 = open(output3_path, 'w')

for index in survey_index_cum_lst:
    survey_question_p30 =' {"Type":"Question","QuestionID":"QID'
    survey_question_p31 = str(index)
    survey_question_p32 = '"},'
    survey_question_y = survey_question_p30+survey_question_p31+survey_question_p32
    result3.write(survey_question_y)
result3.close()

In [54]:
max(survey_index_cum_lst)



487